In [1]:
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving first_500000_rows.csv to first_500000_rows.csv


In [3]:
df = pd.read_csv('/content/first_500000_rows.csv')

In [4]:
column_data = df['dealer_final_value']

print(column_data)


0         24
1         18
2         22
3         18
4         20
          ..
499995    17
499996    19
499997    18
499998    22
499999    20
Name: dealer_final_value, Length: 500000, dtype: object


In [5]:
# Remove any leading/trailing whitespace
column_data = column_data.str.strip()

# Replace any non-numeric characters with empty string
column_data = column_data.str.replace('[^0-9]', '', regex=True)

# Convert to numeric (will result in NaN for values that cannot be converted)
column_data = pd.to_numeric(column_data, errors='coerce')

# Optionally, you can fill NaN values with a specific value (e.g., 0)
# column_data = column_data.fillna(0)

print(column_data)


0         24.0
1         18.0
2         22.0
3         18.0
4         20.0
          ... 
499995    17.0
499996    19.0
499997    18.0
499998    22.0
499999    20.0
Name: dealer_final_value, Length: 500000, dtype: float64


In [6]:
column_data_2 = df['player_final_value']

print(column_data_2)

0         ['BJ']
1           [21]
2           [13]
3           [18]
4           [26]
           ...  
499995      [17]
499996      [14]
499997      [17]
499998      [12]
499999      [20]
Name: player_final_value, Length: 500000, dtype: object


In [7]:
column_data_2 = column_data_2.str.strip()

column_data_2 = column_data_2.str.replace('[^0-9]', '', regex=True)

column_data_2 = pd.to_numeric(column_data_2, errors='coerce')


print(column_data_2)

0          NaN
1         21.0
2         13.0
3         18.0
4         26.0
          ... 
499995    17.0
499996    14.0
499997    17.0
499998    12.0
499999    20.0
Name: player_final_value, Length: 500000, dtype: float64


In [8]:
# Remove rows with missing values in both columns
df_cleaned = df.dropna(subset=['dealer_final_value', 'player_final_value'])

# Explore the cleaned data
print(column_data.describe())
print(column_data.isnull().sum())
print(column_data_2.describe())
print(column_data_2.isnull().sum())

count    476524.000000
mean         20.333702
std           2.580619
min          17.000000
25%          18.000000
50%          20.000000
75%          22.000000
max          26.000000
Name: dealer_final_value, dtype: float64
23476
count    4.761030e+05
mean     8.535148e+03
std      3.803095e+05
min      4.000000e+00
25%      1.600000e+01
50%      1.800000e+01
75%      2.000000e+01
max      2.625252e+07
Name: player_final_value, dtype: float64
23897


In [9]:
# Remove duplicate rows based on all columns
df_no_duplicates = df_cleaned.drop_duplicates()

# Explore the data after removing duplicates
print(df_no_duplicates.shape)  # Check the new shape


(500000, 12)


In [10]:
import numpy as np

# Calculate the IQR for both columns
Q1_dealer = column_data.quantile(0.25)
Q3_dealer = column_data.quantile(0.75)
IQR_dealer = Q3_dealer - Q1_dealer

Q1_player = column_data_2.quantile(0.25)
Q3_player = column_data_2.quantile(0.75)
IQR_player = Q3_player - Q1_player

# Define bounds for outliers
lower_bound_dealer = Q1_dealer - 1.5 * IQR_dealer
upper_bound_dealer = Q3_dealer + 1.5 * IQR_dealer

lower_bound_player = Q1_player - 1.5 * IQR_player
upper_bound_player = Q3_player + 1.5 * IQR_player

# Identify outliers
outliers_dealer = df_no_duplicates[(column_data > upper_bound_dealer) | (column_data < lower_bound_dealer)]
outliers_player = df_no_duplicates[(column_data_2 > upper_bound_player) | (column_data_2 < lower_bound_player)]

# Remove them:
df_no_outliers = df_no_duplicates[
    (column_data <= upper_bound_dealer) & (column_data >= lower_bound_dealer) &
    (column_data_2 <= upper_bound_player) & (column_data_2 >= lower_bound_player)]

# Explore the data after outlier handling
print(df_no_outliers.shape)


(441377, 12)


In [11]:
# Iterate through columns and infer data types
for column in df_cleaned.columns:
  try:
    # Try converting to numeric
    df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors='ignore')
  except:
    pass  # If not numeric, leave as is

# Print the data types of each column after conversion
print(df_cleaned.dtypes)


shoe_id                 int64
cards_remaining         int64
dealer_up               int64
initial_hand           object
dealer_final           object
dealer_final_value     object
player_final           object
player_final_value     object
actions_taken          object
run_count               int64
true_count              int64
win                   float64
dtype: object


<ipython-input-11-50b7b761c77e>:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors='ignore')


In [12]:
new_column_names = {
    'dealer_final_value': 'Dealer_Final_Value',
    'player_final_value': 'Player_Final_Value'
}

df_cleaned = df_cleaned.rename(columns=new_column_names)

print(df_cleaned.columns)


Index(['shoe_id', 'cards_remaining', 'dealer_up', 'initial_hand',
       'dealer_final', 'Dealer_Final_Value', 'player_final',
       'Player_Final_Value', 'actions_taken', 'run_count', 'true_count',
       'win'],
      dtype='object')


In [13]:
df.to_csv('cleaned_data.csv', index=False)

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# 1. Creating dummy features (if needed)
# Example: If you have a categorical feature 'game_type'
# df_no_outliers = pd.get_dummies(df_no_outliers, columns=['game_type'], drop_first=True)


# 2. Scale standardization
# Select numerical features to scale (excluding dummy variables)
numerical_features = ['Dealer_Final_Value', 'Player_Final_Value'] # Example, adjust as needed
scaler = StandardScaler()

# Fit and transform the numerical features
outliers_dealer = df_no_duplicates[(column_data > upper_bound_dealer) | (column_data < lower_bound_dealer)]
outliers_player = df_no_duplicates[(column_data_2 > upper_bound_player) | (column_data_2 < lower_bound_player)]

# Instead of removing all outliers, consider capping them to the IQR bounds:
df_no_outliers = df_no_duplicates.copy()  # Create a copy to avoid modifying the original
# Ensure both columns are numeric before clipping
df_no_outliers['dealer_final_value'] = pd.to_numeric(df_no_outliers['dealer_final_value'], errors='coerce')
df_no_outliers['player_final_value'] = pd.to_numeric(df_no_outliers['player_final_value'], errors='coerce')

# Now apply np.clip
df_no_outliers['dealer_final_value'] = np.clip(df_no_outliers['dealer_final_value'], lower_bound_dealer, upper_bound_dealer)
df_no_outliers['player_final_value'] = np.clip(df_no_outliers['player_final_value'], lower_bound_player, upper_bound_player)
df_no_outliers['dealer_final_value'] = np.clip(df_no_outliers['dealer_final_value'], lower_bound_dealer, upper_bound_dealer)
df_no_outliers['player_final_value'] = np.clip(df_no_outliers['player_final_value'], lower_bound_player, upper_bound_player)

# Explore the data after outlier handling
print(df_no_outliers.shape)


# 3. Split data into training and testing subsets
# Define features (X) and target variable (y)
X = df_no_outliers.drop(['dealer_final_value', 'player_final_value'], axis=1)
y = df_no_outliers['player_final_value']

# Split the data (adjust test_size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shapes after splitting:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

(500000, 12)
Shapes after splitting:
X_train: (400000, 10)
X_test: (100000, 10)
y_train: (400000,)
y_test: (100000,)
